### Importando as bibliotecas

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd


In [38]:
# importar/ visualizar a base de dados
arquivo = 'buscas.xlsx'
tabela = pd.read_excel(arquivo)
display(tabela)


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000.0,3500.0
1,rtx 3060,zota galax,1500.0,4500.0


In [39]:
# criar navegador
driver = webdriver.Chrome()


### Definição das funções de busca do Buscapé

In [40]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
                
    return tem_termos_banidos


def verificar_tem_todos_termos_produto(lista_termos_produto, nome):
    tem_todos_termos_produto = True
    for palavra in lista_termos_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False

    return tem_todos_termos_produto


def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    lista_termos_banidos = termos_banidos.split()
    lista_termos_produto = produto.split()
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    lista_ofertas = []
    
    driver.get('https://www.buscape.com.br/')
    
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__HG105').send_keys(produto, Keys.ENTER)
    while len(driver.find_elements(By.CLASS_NAME, 'Select_Select__1S7HV')) < 1:
        time.sleep(1)
    
    # pegar resultados dos produtos
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    for item in lista_resultados:
        nome = item.find_element(By.CLASS_NAME, 'Text_DesktopLabelSAtLarge__baj_g').text
        nome = nome.lower()
    
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_produto, nome)
         
        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_nome_produto = True 
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = item.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            preco = float(preco) 
           

            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = driver.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '.')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((nome, preco, link))
                
    return lista_ofertas


### Construindo a lista de ofertas

In [41]:
tabela_ofertas = pd.DataFrame()

for linha in tabela.index:
    
    # pesquisar pelo produto
    produto = tabela.loc[linha, 'Nome']
    termos_banidos = tabela.loc[linha, 'Termos banidos']
    preco_minimo = tabela.loc[linha, 'Preço mínimo']
    preco_maximo = tabela.loc[linha, 'Preço máximo']

    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
        
    display(tabela_buscape)
    
display(tabela_ofertas)


,Produto,Preço,Link
0,smartphone apple iphone 12 64gb câmera dupla,3000.0,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 12 usado 64gb câmera d...,3049.0,https://www.buscape.com.br/celular/smartphone-...


,Produto,Preço,Link
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,2349.00,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 3060 12 gb g...,1889.00,https://www.buscape.com.br/placa-de-video/plac...
2,placa de video nvidia geforce rtx 3060 ti 8 gb...,3599.00,https://www.buscape.com.br/placa-de-video/plac...
3,placa de video nvidia geforce rtx 3060 12 gb g...,2554.44,https://www.buscape.com.br/placa-de-video/plac...
4,placa de video rtx 3060 ti dual fan pny nvidia...,2499.99,https://www.buscape.com.br/placa-de-video/plac...
5,"placa de video pny nvidia geforce rtx 3060, 12...",2359.99,https://www.buscape.com.br/placa-de-video/plac...
6,placa de vídeo nvidia msi geforce rtx 3060 ven...,1863.75,https://www.buscape.com.br/placa-de-video/plac...
7,placa de vídeo rtx 3060 asus dual o12g v2 nvid...,2099.99,https://www.buscape.com.br/placa-de-video/plac...
8,placa de vídeo rtx 3060 ventus 2x msi nvidia g...,1999.99,https://www.buscape.com.br/placa-de-video/plac...
9,placa de vídeo rtx 3060 oc edition asus dual g...,2059.99,https://www.buscape.com.br/placa-de-video/plac...


,Produto,Preço,Link
0,smartphone apple iphone 12 64gb câmera dupla,3000.00,https://www.buscape.com.br/celular/smartphone-...
1,smartphone apple iphone 12 usado 64gb câmera d...,3049.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,2349.00,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 3060 12 gb g...,1889.00,https://www.buscape.com.br/placa-de-video/plac...
2,placa de video nvidia geforce rtx 3060 ti 8 gb...,3599.00,https://www.buscape.com.br/placa-de-video/plac...
3,placa de video nvidia geforce rtx 3060 12 gb g...,2554.44,https://www.buscape.com.br/placa-de-video/plac...
4,placa de video rtx 3060 ti dual fan pny nvidia...,2499.99,https://www.buscape.com.br/placa-de-video/plac...
5,"placa de video pny nvidia geforce rtx 3060, 12...",2359.99,https://www.buscape.com.br/placa-de-video/plac...
6,placa de vídeo nvidia msi geforce rtx 3060 ven...,1863.75,https://www.buscape.com.br/placa-de-video/plac...
7,placa de vídeo rtx 3060 asus dual o12g v2 nvid...,2099.99,https://www.buscape.com.br/placa-de-video/plac...


### Exportando para o Excel

In [42]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

### Enviando o e-mail

In [43]:
import win32com.client as win32
if len(tabela_ofertas) > 0:
    outlook = win32.Dispatch('outlook.application')

    mail = outlook.CreateItem(0)
    mail.To = 'seu_email@hotmail.com'
    mail.Subject = 'Produtos encontrado(s) na faixa de preço desejada.'
    mail.HTMLBody = f'''
    <p>Prezados, </p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada.</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att., Rayssa.</p>
    '''
    mail.Send()
driver.quit()